In [3]:
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
from keras.optimizers import Adam 
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers.schedules import ExponentialDecay 
import cv2 
from keras.models import model_from_json 
import numpy as np


In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam


In [4]:
train_data_gen_frustration = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)


In [15]:
import os
import tensorflow as tf
data_dir = 'db_for_cnn/train'
minority_class = 'frustration'

# Crear un ImageDataGenerator para la clase minoritaria con data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Función para contar el número de imágenes en una carpeta
def count_images_in_folder(folder_path):
    return len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])

# Obtener el número de imágenes en cada clase
num_class_0 = count_images_in_folder(os.path.join(data_dir, 'neutral'))
num_class_1 = count_images_in_folder(os.path.join(data_dir, 'frustration'))

# Número de veces que necesitas aumentar las imágenes de la clase minoritaria
augmentation_factor = num_class_0 // num_class_1

# Cargar las imágenes de la clase minoritaria
minority_class_path = os.path.join(data_dir, minority_class)
minority_images = [os.path.join(minority_class_path, fname) for fname in os.listdir(minority_class_path) if os.path.isfile(os.path.join(minority_class_path, fname))]

# Aplicar data augmentation y guardar las imágenes aumentadas
save_to_dir = os.path.join(data_dir, minority_class, 'augmented')
os.makedirs(save_to_dir, exist_ok=True)

for img_path in minority_images:
    img = tf.keras.preprocessing.image.load_img(img_path)
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix='aug', save_format='jpg'):
        i += 1
        if i >= augmentation_factor:
            break

In [16]:
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_data_gen.flow_from_directory(
    data_dir,
    target_size=(64,64),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
) 

Found 129369 images belonging to 2 classes.


In [20]:
train_generator

In [22]:
validation_generator = validation_data_gen.flow_from_directory(
    'db_for_cnn/dev',
    target_size=(64,64),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 39780 images belonging to 2 classes.


In [23]:
test = test_data_gen.flow_from_directory(
    'db_for_cnn/test',
    target_size=(64,64),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 39840 images belonging to 2 classes.


In [21]:
emotion_model = Sequential() 

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                        input_shape=(64, 64, 1))) 
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Dropout(0.2)) 

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Dropout(0.25)) 

emotion_model.add(Flatten()) 
emotion_model.add(Dense(216, activation='relu')) 
emotion_model.add(Dropout(0.2)) 
emotion_model.add(Dense(2, activation='softmax')) 

emotion_model.summary() 

cv2.ocl.setUseOpenCL(False) 

initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000,decay_rate=0.96) 

optimizer = Adam(learning_rate=lr_schedule) 

emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 30, 30, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                    

In [24]:
emotion_model_info = emotion_model.fit_generator( 
        train_generator, 
        steps_per_epoch=28709 // 64, 
        epochs=30, 
        validation_data=validation_generator, 
        validation_steps=7178 // 64)

C:\Users\bl4ck\AppData\Local\Temp\ipykernel_31716\2100404603.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/30
448/448 [==============================] - 156s 348ms/step - loss: 0.3825 - accuracy: 0.8623 - val_loss: 0.3147 - val_accuracy: 0.8955
Epoch 2/30
448/448 [==============================] - 155s 346ms/step - loss: 0.2999 - accuracy: 0.8913 - val_loss: 0.2824 - val_accuracy: 0.8991
Epoch 3/30
448/448 [==============================] - 154s 344ms/step - loss: 0.2657 - accuracy: 0.9007 - val_loss: 0.2652 - val_accuracy: 0.9008
Epoch 4/30
448/448 [==============================] - 156s 349ms/step - loss: 0.2398 - accuracy: 0.9079 - val_loss: 0.2392 - val_accuracy: 0.9061
Epoch 5/30
448/448 [==============================] - 159s 354ms/step - loss: 0.2158 - accuracy: 0.9159 - val_loss: 0.2302 - val_accuracy: 0.9111
Epoch 6/30
448/448 [==============================] - 157s 350ms/step - loss: 0.2111 - accuracy: 0.9182 - val_loss: 0.2297 - val_accuracy: 0.9071
Epoch 7/30
448/448 [==============================] - 156s 349ms/step - loss: 0.1928 - accuracy: 0.9242 - val_loss: 0.2073 -

KeyboardInterrupt: 

: 